# Step 2 · Normalize SIATA feeds

This notebook flattens the historic and current precipitation JSON feeds into a common schema ready for loading into PostgreSQL. It also persists intermediate artifacts for inspection.


## 1. Setup & paths


In [1]:

from __future__ import annotations
import json
import os
from pathlib import Path
from typing import Any, Dict, Iterable, List

import pandas as pd
from dotenv import load_dotenv

load_dotenv(dotenv_path=Path('..') / '.env', override=True)

BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
PROCESSED_DIR = BASE_DIR / 'data' / 'processed'
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

HISTORIC_PATH = RAW_DIR / 'historic_precipitation.json'
CURRENT_PATH = RAW_DIR / 'current_precipitation.json'

HISTORIC_PATH, CURRENT_PATH


(PosixPath('/home/zerotwo/iot/Siata-Contamination-Viewer/data/raw/historic_precipitation.json'),
 PosixPath('/home/zerotwo/iot/Siata-Contamination-Viewer/data/raw/current_precipitation.json'))

## 2. Load raw feeds

In [2]:

with HISTORIC_PATH.open() as f:
    historic_raw = json.load(f)

with CURRENT_PATH.open() as f:
    current_raw = json.load(f)

len(historic_raw), len(current_raw['estaciones'])


(13, 226)

## 3. Normalize sensor metadata

We create a unified `sensors` table with deterministic IDs. Historic Vaisala stations are prefixed with `vaisala_`, current pluvio stations with `pluvio_`.


In [3]:

def to_float(value: Any) -> float | None:
    try:
        if value is None:
            return None
        if isinstance(value, str) and not value.strip():
            return None
        return float(value)
    except (TypeError, ValueError):
        return None


def normalize_historic_sensors(raw: Iterable[Dict[str, Any]]) -> pd.DataFrame:
    records: List[Dict[str, Any]] = []
    for item in raw:
        codigo = str(item.get('codigo'))
        sensor_id = f"vaisala_{codigo}"
        records.append({
            'id': sensor_id,
            'provider_id': codigo,
            'name': item.get('nombre'),
            'lat': to_float(item.get('latitud')),
            'lon': to_float(item.get('longitud')),
            'elevation_m': None,
            'city': item.get('ciudad'),
            'subbasin': item.get('subcuenca'),
            'barrio': None,
            'metadata': {
                'source': 'historic',
                'raw': {
                    'latitud': item.get('latitud'),
                    'longitud': item.get('longitud'),
                    'subcuenca': item.get('subcuenca'),
                }
            }
        })
    return pd.DataFrame.from_records(records)


def normalize_current_sensors(raw: Iterable[Dict[str, Any]]) -> pd.DataFrame:
    records: List[Dict[str, Any]] = []
    for item in raw:
        codigo = str(item.get('codigo'))
        sensor_id = f"pluvio_{codigo}"
        records.append({
            'id': sensor_id,
            'provider_id': codigo,
            'name': item.get('nombre'),
            'lat': to_float(item.get('latitud')),
            'lon': to_float(item.get('longitud')),
            'elevation_m': None,
            'city': item.get('ciudad'),
            'subbasin': item.get('subcuenca'),
            'barrio': item.get('barrio'),
            'metadata': {
                'source': 'current',
                'comuna': item.get('comuna'),
            }
        })
    return pd.DataFrame.from_records(records)

historic_sensors_df = normalize_historic_sensors(historic_raw)
current_sensors_df = normalize_current_sensors(current_raw['estaciones'])

sensors_df = (
    pd.concat([historic_sensors_df, current_sensors_df], ignore_index=True)
    .drop_duplicates(subset=['id'])
)

print('Historic sensors:', len(historic_sensors_df))
print('Current sensors:', len(current_sensors_df))
print('Combined sensors:', len(sensors_df))
sensors_df.head()


Historic sensors: 13
Current sensors: 226
Combined sensors: 239


,id,provider_id,name,lat,lon,elevation_m,city,subbasin,barrio,metadata
0,vaisala_59,59,ISAGEN,6.213940,-75.558970,None,Medellin,Q. La Presidenta,None,"{'source': 'historic', 'raw': {'latitud': '6.2..."
1,vaisala_68,68,Jardin Botanico,6.269250,-75.562910,None,Medellin,Q. La Chorrera,None,"{'source': 'historic', 'raw': {'latitud': '6.2..."
2,vaisala_73,73,Ciudadela Educativa La Vida,6.345343,-75.504722,None,Copacabana,R. Aburra-Medellin,None,"{'source': 'historic', 'raw': {'latitud': '6.3..."
3,vaisala_82,82,I.E Manuel Jose Caicedo,6.434950,-75.326646,None,Barbosa,Q. Agua Caliente,None,"{'source': 'historic', 'raw': {'latitud': '6.4..."
4,vaisala_83,83,Centro de Salud San Javier La Loma - Meteorolo...,6.271891,-75.631123,None,Medellin,Q. La Iguana,None,"{'source': 'historic', 'raw': {'latitud': '6.2..."


## 4. Flatten measurement records

Historic feed contains high-frequency minute data per station. The current feed only reports the latest reading per station; we tag those with the retrieval timestamp.


In [4]:

from datetime import datetime, timezone


def normalize_historic_measurements(raw: Iterable[Dict[str, Any]]) -> pd.DataFrame:
    records: List[Dict[str, Any]] = []
    for item in raw:
        sensor_id = f"vaisala_{item.get('codigo')}"
        for slot in item.get('datos', []):
            ts_str = slot.get('fecha')
            try:
                ts = pd.to_datetime(ts_str, utc=True)
            except Exception:
                ts = pd.NaT
            for var_entry in slot.get('datos', []):
                records.append({
                    'sensor_id': sensor_id,
                    'ts': ts,
                    'value_mm': to_float(var_entry.get('valor')),
                    'quality': to_float(var_entry.get('calidad')),
                    'variable': var_entry.get('variableConsulta'),
                    'source': 'historic'
                })
    df = pd.DataFrame.from_records(records)
    # Filter to precipitation variable (defensive if other variables appear)
    if not df.empty:
        df = df[df['variable'] == 'precipitacion']
    return df


def normalize_current_measurements(raw: Iterable[Dict[str, Any]], retrieval_ts: datetime) -> pd.DataFrame:
    records: List[Dict[str, Any]] = []
    for item in raw:
        sensor_id = f"pluvio_{item.get('codigo')}"
        value = to_float(item.get('valor'))
        if value is not None and value < 0:
            # Current feed uses -999 for missing values
            value = None
        records.append({
            'sensor_id': sensor_id,
            'ts': pd.Timestamp(retrieval_ts),
            'value_mm': value,
            'quality': None,
            'variable': 'precipitacion',
            'source': 'current'
        })
    return pd.DataFrame.from_records(records)

historic_measurements_df = normalize_historic_measurements(historic_raw)
current_measurements_df = normalize_current_measurements(
    current_raw['estaciones'],
    retrieval_ts=datetime.now(timezone.utc).replace(microsecond=0)
)

print('Historic measurements:', len(historic_measurements_df))
print('Current measurements:', len(current_measurements_df))

combined_measurements_df = pd.concat(
    [historic_measurements_df, current_measurements_df],
    ignore_index=True
)
combined_measurements_df.head()


Historic measurements: 561613
Current measurements: 226


/tmp/ipykernel_180064/3622080106.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_measurements_df = pd.concat(


,sensor_id,ts,value_mm,quality,variable,source
0,vaisala_59,2022-06-07 17:57:00+00:00,0.0,1.0,precipitacion,historic
1,vaisala_59,2022-06-07 17:58:00+00:00,0.0,1.0,precipitacion,historic
2,vaisala_59,2022-06-07 17:59:00+00:00,0.0,1.0,precipitacion,historic
3,vaisala_59,2022-06-07 18:00:00+00:00,0.0,1.0,precipitacion,historic
4,vaisala_59,2022-06-07 18:01:00+00:00,0.0,1.0,precipitacion,historic


### Quick inspection

In [5]:

combined_measurements_df.describe(include='all')


,sensor_id,ts,value_mm,quality,variable,source
count,561839,561839,561794.000000,561613.000000,561839,561839
unique,239,NaN,NaN,NaN,1,2
top,vaisala_59,NaN,NaN,NaN,precipitacion,historic
freq,43201,NaN,NaN,NaN,561839,561613
mean,NaN,2022-06-23 05:28:46.474669312+00:00,-862.432442,120.838921,NaN,NaN
min,NaN,2022-06-07 17:57:00+00:00,-9999.000000,1.000000,NaN,NaN
25%,NaN,2022-06-15 06:01:00+00:00,0.000000,1.000000,NaN,NaN
50%,NaN,2022-06-22 18:06:00+00:00,0.000000,1.000000,NaN,NaN
75%,NaN,2022-06-30 06:10:00+00:00,0.000000,1.000000,NaN,NaN
max,NaN,2025-09-29 00:37:47+00:00,2.760000,15345.000000,NaN,NaN


## 5. Persist normalized artifacts

We save the normalized outputs for downstream services and for verifying the transformation.


In [6]:

sensors_output = PROCESSED_DIR / 'sensors_normalized.parquet'
measurements_output = PROCESSED_DIR / 'measurements_normalized.parquet'

sensors_df.to_parquet(sensors_output, index=False)
combined_measurements_df.to_parquet(measurements_output, index=False)

sensors_output, measurements_output


(PosixPath('/home/zerotwo/iot/Siata-Contamination-Viewer/data/processed/sensors_normalized.parquet'),
 PosixPath('/home/zerotwo/iot/Siata-Contamination-Viewer/data/processed/measurements_normalized.parquet'))

## 6. Load into PostgreSQ

The following cell performs idempotent upserts into the schema defined in `db/schema.sql`. Configure `DATABASE_URL` in `.env` before running. The helper uses PostgreSQL `ON CONFLICT` to avoid duplicating records.


In [ ]:

import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import insert


DATABASE_URL = os.getenv('DATABASE_URL')
if not DATABASE_URL:
    raise RuntimeError('DATABASE_URL not set; update .env before loading data.')

engine = sa.create_engine(DATABASE_URL, pool_pre_ping=True, future=True)

sensors_records = sensors_df.to_dict(orient='records')
measurements_records = combined_measurements_df.to_dict(orient='records')

metadata = sa.MetaData()
sensors_table = sa.Table('sensors', metadata, autoload_with=engine)
raw_measurements_table = sa.Table('raw_measurements', metadata, autoload_with=engine)

with engine.begin() as conn:
    if sensors_records:
        stmt = insert(sensors_table).values(sensors_records)
        update_cols = {col.name: col for col in stmt.excluded if col.name not in ('id',)}
        conn.execute(stmt.on_conflict_do_update(index_elements=['id'], set_=update_cols))

    if measurements_records:
        stmt = insert(raw_measurements_table).values(measurements_records)
        conn.execute(
            stmt.on_conflict_do_update(
                index_elements=['sensor_id', 'ts'],
                set_={
                    'value_mm': stmt.excluded.value_mm,
                    'quality': stmt.excluded.quality,
                    'variable': stmt.excluded.variable,
                    'source': stmt.excluded.source,
                    'updated_at': sa.func.now()
                }
            )
        )
print('Load completed')
